In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv("train.csv",encoding = "utf8",engine = "python", index_col='ID')
test = pd.read_csv("20190513_test.csv",encoding = "utf8",engine = "python",index_col='ID')
old_test = pd.read_csv("20190506_test.csv",encoding = "utf8",engine = "python",index_col='ID')
train["label"] = train["label"].map(lambda x: 1 if x=="Positive" else 0)

In [3]:
train.head()

,review,label
ID,,
1,Jo bhi ap se tou behtar hoon,0
2,ya Allah meri sister Affia ki madad farma,1
3,Yeh khud chahta a is umar main shadi krna. ha...,0
4,Tc ? Apky mun xe exe alfax achy nae lgty 😒💃,0
5,Good,1


In [4]:
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
import re
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

In [5]:
def removeEmoji(sentence):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', sentence)
#https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
#Whether remove or not,I think both are fine

In [6]:
train["review"] = train["review"].map(lambda x:removeEmoji(x))
old_test["review"] = old_test["review"].map(lambda x:removeEmoji(x))
test["review"] = test["review"].map(lambda x:removeEmoji(x))
train.head()

,review,label
ID,,
1,Jo bhi ap se tou behtar hoon,0
2,ya Allah meri sister Affia ki madad farma,1
3,Yeh khud chahta a is umar main shadi krna. ha...,0
4,Tc ? Apky mun xe exe alfax achy nae lgty,0
5,Good,1


In [7]:
corpus = list(train["review"])+list(old_test["review"])
Vectorize =TfidfVectorizer(token_pattern=r'\w{1,}', max_features=35000, ngram_range=(1, 1))
Vectorize.fit(corpus)
X = Vectorize.transform(train["review"])
y = train['label'] 
X_test = Vectorize.transform(test["review"])
print(X.shape)
print(y.shape)

(6331, 21904)
(6331,)


In [8]:
#First vectorize then select features
K = 6650
model = SelectKBest(chi2,k=K)
X_new = model.fit_transform(X,y)
X_test_new = model.transform(X_test)
print(X_new.shape)
print(X_test_new.shape)
#I guess it won't work well

(6331, 6650)
(2712, 6650)


In [10]:
#SVM
K = 10
seed =20
oof_preds = np.zeros((X.shape[0]))
test_preds = np.zeros((X_test.shape[0]))
kf = KFold(n_splits= K)
for train_ind,val_ind in kf.split(X):
  clf = MultinomialNB(alpha=0.21)
# 0.21 0.19
  clf.fit(X_new[train_ind],y[train_ind])
  oof_preds[val_ind] += clf.predict_proba(X_new[val_ind])[:,1]
  test_preds += clf.predict_proba(X_test_new)[:,1]
test_preds/=K
score = roc_auc_score(y>0.5,oof_preds)
score

0.9365354302156036

In [11]:
sample = pd.read_csv("20190506_test.csv")
submission = pd.DataFrame(columns=["ID","Pred"])
submission['Pred'] = test_preds
submission['ID'] = sample["ID"]
submission.head()

,ID,Pred
0,1,0.951247
1,2,0.858817
2,3,0.810539
3,4,0.463836
4,5,0.912071


In [222]:
score = str(score)
submission.to_csv(score+'submission.csv', index=False)